# Using Manim to Animate and Visualise Part of The Simplex Method 

The [3Blue1Brown](https://www.youtube.com/c/3blue1brown) YouTube channel explains complex and important Mathematical in a simple and efficient manner utilizing beautiful Mathematical animations. 

The creator, Grant Sanderson, made this possible by creating his own graphics library / Mathematical Animation Engine in Python called [Manim](https://docs.manim.community/en/stable/index.html). It is now a massive open source project and allows anyone to start creating similar animations. Inspired by his channel, I will animate and draw out a convex polytope and see how the objective function values change as the Simplex Algorithm iterates over it.  

We shall find the feasible region of a 2D linear programming problem and create an animation, that enumerates through the feasible region and finds the maximum value for the objective function. 

- Suppose we have the following LP problem:

- maximize $180x+200y$
  - subject to: <br>
  - $5x+4y \leq 80$ <br>
  - $10x+20y \leq 200$ <br>
  - $x, y \geq 0$

- If we are to solve it, we get that the optimal value is $3066.6666667$ with $x = 13.33$ and $y = 3.33$
- We will now graph the linear equations and iterate over the convex polytope. We will see how the objective function changes as it moves across the polytope and arrives at an optimal value. 

In [1]:
# imports the relevant Manim library
from manim import *


Manim Community v0.17.1

### There is quite a lot happening in the following scene, I will try to break down what I am doing step by step in the code comments

In [15]:
%%manim -qm -v WARNING Graphing2

class Graphing2(Scene):
    def construct(self):

      # Mobject Creation and Scene Setup

      # create and position the Obj. function in Latex to write out
        obj_func = Tex("Objective Function:").to_edge(UL, buff = 0.5)
        obj_func_val = MathTex("180x+200y = ").next_to(obj_func)
      

        
      # creates the axes to plot the linear functions on
      # we see the domain and range is from (-10, 25) going up in increments of 2.5
        ax = Axes(
            x_range=[-10, 25, 2.5], y_range=[-10, 25, 2.5], axis_config={"include_tip": True}
        )
        
      # these dots move to the points where 5x+4y <= 80 and 10x+200y <= 200
      # intersect. Also moves to the points where x = 0 for both functions 
        dot1 = Dot(0.48).move_to(ax.coords_to_point(0, 10))
        dot2 = Dot(0.48).move_to(ax.coords_to_point(13.33, 3.33))
        dot3 = Dot(0.48).move_to(ax.coords_to_point(16, 0))
        dot4 = Dot(0.48).move_to(ax.coords_to_point(0, 0))
        
      # simply created 4 text labels for the 4 points of intersection 
        origin_text1 = Text('(0, 10)', font_size = 15).next_to(dot1, LEFT)
        origin_text2 = Text('(13.333, 3.333)', font_size = 15).next_to(dot2, LEFT)
        origin_text3 = Text('(16, 0)', font_size = 15).next_to(dot3, UP)
        origin_text4 = Text('(0, 0)', font_size = 15).next_to(dot4, UP)
        
      # plot both linear functions and provide them with labels 
        line_1 = ax.plot(lambda x: 20-5*x/4, color=BLUE_C)
        line_2 = ax.plot(lambda x: 10-0.5*x,color=GREEN_B)
        line_1_label = MathTex("5x+4y < 80").scale(0.6).next_to(line_1, UR, buff = -4.6)
        line_2_label = MathTex("10x+20y < 200").scale(0.6).next_to(line_2, UP, buff = 0.5)
        
      # since we are graphing inequalities, we use the 'get_area' function to find 
      # and color in the area below 'line_1' and 'line_2' 
        area1 = ax.get_area(line_1, [-10, 25], color=BLUE, opacity=0.5)
        area2 = ax.get_area(line_2, [-10, 25], color=GREEN, opacity=0.5)
        
      # -------------------------------------------- 
      # Variable Tracking and Animation Logic 
      # as we saw in Artifact 1, to show the function value changing we need a 
      # function definition and value tracker 
        y = ValueTracker(10)

        def func(x):
            return 10-0.5*x 

      # First Simplex Region Iteration 
      # since x and y are >= 0, for the first simplex region iteration, we go 
      # from (0, 0) to (0, 10) since rate of increase is higher towards y
      # the objective function value changes accordingly and is shown on screen

        temp = ValueTracker(0)
        def fs1(x):
            return 0
        
        initial_point1 = [ax.coords_to_point(fs1(temp.get_value()), temp.get_value())]
        fs1dot = Dot(point=initial_point1)
        fs1dot.add_updater(lambda x: x.move_to(ax.c2p(fs1(temp.get_value()), temp.get_value())))
        num1 = always_redraw(lambda: DecimalNumber().set_value(round(200*temp.get_value())).next_to(obj_func_val))
        
       # Second Simplex Region Iteration 
       # Go along the function 'func' as the dot moves from (0, 10) to (13.333, 3.333) 
        t = ValueTracker(0)
        num2 = always_redraw(lambda: DecimalNumber().set_value(round(180*t.get_value()+200*y.get_value())).next_to(obj_func_val))
        initial_point = [ax.coords_to_point(t.get_value(), func(t.get_value()))]
       # dot turns green to showcase that we have found the max value   
        fs2dot = Dot(point=initial_point, color = PURE_GREEN)
        fs2dot.add_updater(lambda x: x.move_to(ax.c2p(t.get_value(), func(t.get_value()))))
        

       # Third Simplex Region Iteration 
       # at this point, we have found the max value however goes through third 
       # iteration to showcase smaller value
       # Go along the function 'fs2' as the dot moves from (13.333, 3.333) to (16, 0) 

        temp2 = ValueTracker(13.333)
        y2 = ValueTracker(3.333)
        def fs2(x):
            return 20-1.25*x

        initial_point3 = [ax.coords_to_point(temp2.get_value(), fs2(temp2.get_value()))]
        
        # dot turns red since less than max value
        fs3dot = Dot(point=initial_point, color = RED)
        fs3dot.add_updater(lambda x: x.move_to(ax.c2p(temp2.get_value(), fs2(temp2.get_value()))))
        # dot turns yellow and scales upwards to show max value of obj function has been found 
        fs4dot = Dot(point=ax.coords_to_point(13.333, 3.333), color = YELLOW)
        num3 = always_redraw(lambda: DecimalNumber().set_value(round(180*temp2.get_value()+200*y2.get_value())).next_to(obj_func_val))
        

        # Grouping some of the dot and label mobjects together for ease of readibility

        dotobjects = [FadeIn(dot1), FadeIn(origin_text1), FadeIn(dot2), 
                      FadeIn(origin_text2), FadeIn(dot3), FadeIn(origin_text3), 
                      FadeIn(dot4), FadeIn(origin_text4)]
        
        # arrays for scaling the yellow dot to a certain degree
        arr1 = [2, 0.5, 2, 0.5, 2, 0.5]
        arr2 = np.linspace(1, 2, 11)

        # Animating all the mobjects 
        
        # write out the function 
        self.play(Write(obj_func))
        self.play(Write(obj_func_val))
        # add the axes and linear constraints
        self.add(ax)
        self.play(FadeIn(line_1))
        self.play(FadeIn(line_2))
        self.add(line_1_label, line_2_label)
        self.wait(3)
        # shade the areas underneath both functions 
        self.play(FadeIn(area1))
        self.play(FadeIn(area2))
        # fade in all the intersection points subsequently, one after another
        self.play(AnimationGroup(*dotobjects, lag_ratio = 0.4))
        self.wait(4)
        # show dot moving from (0, 0) to (0, 10)
        self.add(fs1dot, num1)
        self.wait(2)
        # show objective function value changing as dot moves 
        self.play(temp.animate.set_value(10), run_time = 2.5, rate_func = smooth)
        self.wait(2)
        self.remove(fs1dot)
        self.remove(num1)
        self.add(fs2dot, num2)
        # show dot moving from (0, 10) to (13.333, 3.333)
        self.play(t.animate.set_value(13.333), y.animate.set_value(3.333), run_time = 2.5, rate_func = smooth)
        self.wait(1.5)
        self.remove(num2)
        self.add(fs3dot, num3)
        # show dot moving from (13.333, 3.333) to (16, 0) 
        self.play(temp2.animate.set_value(16), y2.animate.set_value(0), run_time = 2.5, rate_func = smooth)
        self.wait(1.5)
       
        self.remove(num3)
        self.add(num2)
        self.add(fs4dot)
        for x in arr1:
            self.play(ScaleInPlace(fs4dot, x))
        self.wait()
        
       

The final max value is scene to be approximately 3067 which is approximately equivalent to 3066.6666667. 